In [5]:
using Distributed

In [170]:
include("./mps.jl");

In [171]:
include("./mpo.jl");

In [172]:
include("./dmrg.jl");

# Test DMRG

In [5]:
cols=3
rows=3
config=zeros(Int,rows,cols)
k=0
for j in 1:cols
    for i in 1:rows
        k+=1
        config[i,j]=k
    end
end
config

3×3 Matrix{Int64}:
 1  4  7
 2  5  8
 3  6  9

In [6]:
d=2
chi=10
chimax=64
sweeps=10
J1=1.
J2=0.5
M=MPO()
Initialize!("J1-J2_ITensors",M,J1,J2,cols,rows,config);

In [7]:
ψ=MPS()
Initialize!(ψ,d,chi,cols*rows)
two_sites_dmrg!(ψ,M,sweeps,chimax)

Done! Energy= -15.837359989589917; Variance: -5.684341886080802e-14


# Test RUC

In [173]:
include("./RUC.jl")
include("./trotter_ev.jl")

trotter_swipe_left (generic function with 1 method)

In [181]:
ψ=MPS()
N=20
d=2
Initialize!("product_state",ψ,repeat(["up","down"],N))

"Product state: [\"up\", \"down\", \"up\", \"down\", \"up\", \"down\", \"up\", \"down\", \"up\", \"down\", \"up\", \"down\", \"up\", \"down\", \"up\", \"down\", \"up\", \"down\", \"up\", \"down\", \"up\", \"down\", \"up\", \"down\", \"up\", \"down\", \"up\", \"down\", \"up\", \"down\", \"up\", \"down\", \"up\", \"down\", \"up\", \"down\", \"up\", \"down\", \"up\", \"down\"]"

In [182]:
Ulist=RUC_layer(ψ.N);

In [ ]:
sweeps=1
ψt=copy(ψ)
for i in 1:10
    Ulist=RUC_layer(ψt.N);
    trotter_evolution!(ψt,Ulist,sweeps)
end

In [113]:
ψt

MPS(Dict{Any, Any}(5 => [0.3032555364890871 - 0.5202572411184291im -0.006282315217647806 - 0.11834562782267187im; -0.058933043353860695 - 0.16654243889320322im 0.16656506782415317 - 0.08801256364850481im; … ; 0.0005909528629555946 + 0.021836518503349014im -0.14306987300132099 - 0.01695278501210143im; -0.04454084209515638 + 0.03363854436688433im 0.03299097700100173 + 0.06789170631067008im;;; 0.005605850757173018 + 0.06404677099155211im -0.20964307552518047 - 0.12763409057014763im; -0.09209952634516905 - 0.27924428236455434im 0.06072086605205722 + 0.24218674772722665im; … ; 0.056737921480988524 - 0.1648296181895739im 0.03776260047447078 - 0.0013396556379491765im; 0.09291282564833558 + 0.08409210840438644im -0.07539853102746064 - 0.11804952401939907im;;; 0.04010233288669712 + 0.032703190029253044im 0.1861820872769097 + 0.2650111301972039im; -0.2512867886103585 + 0.011298431952289145im -0.4363088112817243 - 0.10574464351016653im; … ; 0.030005302873020556 + 0.022679254763977977im 0.00154725

In [29]:
function CUE(nh)
  U = (randn(nh,nh)+im*randn(nh,nh))/sqrt(2)
  q,r = qr(U)
  d = r[diagind(r)]
  ph = d./abs.(d)
  U = ph.*q
  return U
end

CUE (generic function with 1 method)

In [38]:
U=CUE(4)

4×4 Matrix{ComplexF64}:
 -0.19886-0.190806im  -0.258279-0.0216704im  …  -0.492268-0.0162863im
  0.29973+0.624629im  -0.207544+0.174771im      -0.566773-0.292581im
 0.437164+0.137235im   0.379132+0.309515im       0.360069-0.359058im
 -0.46323-0.139716im   0.483254+0.621392im      -0.265602-0.146474im

In [39]:
@show U*U'

U * U' = ComplexF64[0.9999999999999998 + 0.0im 2.498001805406602e-16 - 2.220446049250313e-16im -1.249000902703301e-16 + 8.326672684688674e-17im 1.1102230246251565e-16 + 2.0122792321330962e-16im; 2.498001805406602e-16 + 2.220446049250313e-16im 0.9999999999999998 + 0.0im -8.326672684688674e-17 + 2.7755575615628914e-17im 1.3877787807814457e-17 + 2.220446049250313e-16im; -1.249000902703301e-16 - 8.326672684688674e-17im -8.326672684688674e-17 - 2.7755575615628914e-17im 1.0000000000000002 + 0.0im 0.0 - 1.3877787807814457e-17im; 1.1102230246251565e-16 - 2.0122792321330962e-16im 1.3877787807814457e-17 - 2.220446049250313e-16im 0.0 + 1.3877787807814457e-17im 1.0000000000000002 + 0.0im]


4×4 Matrix{ComplexF64}:
         1.0+0.0im          …  1.11022e-16+2.01228e-16im
   2.498e-16+2.22045e-16im     1.38778e-17+2.22045e-16im
  -1.249e-16-8.32667e-17im             0.0-1.38778e-17im
 1.11022e-16-2.01228e-16im             1.0+0.0im

In [75]:
Us=reshape(U,(2,2,2,2));
Usd=reshape(U',(2,2,2,2));
L=ones(1,1)
A=ψ.data[1]
B=ψ.data[2]
@tensor θ[:]:=A[-1,-2,1]*B[1,-3,-4]

1×2×2×1 Array{ComplexF64, 4}:
[:, :, 1, 1] =
 -0.444789+0.0im  -0.22091+0.0im

[:, :, 2, 1] =
 -0.162551+0.0im  -0.0807329+0.0im

In [79]:
@tensor θt[:]:=Us[1,2,-2,-3]*θ[-1,1,2,-4]

1×2×2×1 Array{ComplexF64, 4}:
[:, :, 1, 1] =
 -0.011426-0.0641465im  0.0600857-0.129449im

[:, :, 2, 1] =
 -0.256476-0.273328im  0.307074+0.142068im

1×2×2×1 Array{ComplexF64, 4}:
[:, :, 1, 1] =
 -0.011426-0.0641465im  0.0600857-0.129449im

[:, :, 2, 1] =
 -0.256476-0.273328im  0.307074+0.142068im

In [70]:
st=size(temp2)
temp3=reshape(temp2,(st[1]*st[2],st[3]*st[4]))

2×2 Matrix{ComplexF64}:
 -0.011426-0.0641465im  -0.256476-0.273328im
 0.0600857-0.129449im    0.307074+0.142068im

In [71]:
A,S,B = svd(temp3,full=false,alg=LinearAlgebra.QRIteration())

SVD{ComplexF64, Float64, Matrix{ComplexF64}, Vector{Float64}}
U factor:
2×2 Matrix{ComplexF64}:
  0.707886-0.138358im   -0.378201+0.580274im
 -0.555214+0.414116im  -0.0933471+0.715215im
singular values:
2-element Vector{Float64}:
 0.5117536337965988
 0.13299304095715803
Vt factor:
2×2 Matrix{ComplexF64}:
 -0.168402+0.0im  -0.499064-0.850045im
 -0.985718-0.0im  0.0852609+0.145223im

In [ ]:
Lambda=

In [73]:
reshape(A,(1,))

2×2 Matrix{ComplexF64}:
  0.707886-0.138358im   -0.378201+0.580274im
 -0.555214+0.414116im  -0.0933471+0.715215im

In [74]:
ones(1,1)

1×1 Matrix{Float64}:
 1.0